In [ ]:
%load_ext autoreload
%autoreload 2

# TruLens for LLMs: Quickstart

In this quickstart you will create a simple LLM Chain and learn how to log it and get feedback on an LLM response.

Note: If you haven't already, make sure to set up your local .env file with your OpenAI and Huggingface Keys. Your .env file should be in the same directory that you run this notebook. If you need help, take a look at the [.env.example](https://github.com/truera/trulens_private/blob/e8b11c4689e644687d2eafe09d90d8d7774b581c/trulens_eval/trulens_eval/.env.example#L4)

## Import from LangChain and TruLens

In [ ]:
from IPython.display import JSON

# imports from langchain to build app
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate, PromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate

# imports from trulens to log and get feedback on chain
from trulens_eval import TruChain, Feedback, OpenAI, Huggingface, db, tru
# from trulens_eval import tru_chain
# from trulens_eval.tru_chain import TruChain
# from trulens_eval.tru_feedback import Feedback, Huggingface, OpenAI
from trulens_eval.keys import *

## Create Simple LLM Application

This example uses a LangChain framework and OpenAI LLM

In [ ]:
full_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template=
        "Provide a helpful response with relevant background information for the following: {prompt}",
        input_variables=["prompt"],
    )
)

chat_prompt_template = ChatPromptTemplate.from_messages([full_prompt])

chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.9)

chain = LLMChain(llm=chat, prompt=chat_prompt_template, verbose=True)

## Send your first request to your new app, asking how to adopt a dog in spanish

In [ ]:
prompt_input = 'Como adopto un perro?'
gpt3_response = chain(prompt_input)

display(gpt3_response)

# Instrument chain for logging with TruLens

In [ ]:
truchain: TruChain = TruChain(chain, chain_id='Chain1_ChatApplication')

In [ ]:
# Instrumented chain can operate like the original:

gpt3_response = truchain(prompt_input)

display(gpt3_response)

In [ ]:
# But can also produce a log or "record" of the execution of the chain:

gpt3_response, record = truchain.call_with_record(prompt_input)

JSON(record)

In [ ]:
# We can log the records but first we need to log the chain itself:

tru.add_chain(chain_json=truchain.json)

In [ ]:
# Now the record:

record_id = tru.add_record(
    prompt=prompt_input, # prompt input
    response=gpt3_response['text'], # LLM response
    record_json=record # record is returned by the TruChain wrapper
)

In [ ]:
# Initialize Huggingface-based feedback function collection class:
hugs = Huggingface()

# Define a language match feedback function using HuggingFace.
f_lang_match = Feedback(hugs.language_match).on(text1="prompt", text2="response")

# This might take a moment if the public api needs to load the language model used in the feedback function:
feedback_result = f_lang_match.run_on_record(chain_json=truchain.json, record_json=record)

In [ ]:
JSON(feedback_result)

In [ ]:
# Alternatively, run a collection of feedback functions:

feedback_results = tru.run_feedback_functions(
    record_json=record,
    feedback_functions=[f_lang_match]
)

In [ ]:
feedback_results

In [ ]:
# These can be logged:

for result in feedback_results:
    tru.add_feedback(result)

## Run the TruLens dashboard to explore the quality of your LLM chain

In [ ]:
tru.run_dashboard() # open a local streamlit app to explore

## Automatic Logging

The above logging and feedback function evaluation steps can be done by TruChain.

In [22]:
truchain: TruChain = TruChain(
    chain,
    chain_id='Chain1_ChatApplication',
    feedbacks=[f_lang_match],
    db=db
)

# Note: providing `db: TruDB` causes the above constructor to log the wrapped chain in the database specified.

# Note: any `feedbacks` specified here will be evaluated and logged whenever the chain is used.

In [23]:
truchain("How are you?")



> Entering new LLMChain chain...
Prompt after formatting:
Human: Provide a helpful response with relevant background information for the following: How are you?

> Finished chain.


{'prompt': 'How are you?',
 'text': 'As an AI language model, I do not have the ability to feel emotions or physical sensations. However, I am always ready and available to assist you in any way I can. Is there anything specific you need help with today?'}